In [7]:
import pandas as pd
import pandas as pd
import clickhouse_driver as ch
from hw import HoltWinters

df = pd.read_csv('test.csv').drop('Unnamed: 0', axis=1)

In [1]:
import pandas as pd
import clickhouse_driver as ch
from hw import HoltWinters

with open('./queries/q.sql', 'r') as f:
    q = f.read()
q=q.replace('${NAME}', 'RU | Гидрактив')
sqlCH = ch.Client(host="192.168.235.72",
                    user="valferov",
                    password="bUfMupl1kDgK1337")

df = pd.DataFrame(sqlCH.execute(q))

In [4]:
df

,0,1,2
0,2022-07-27 10:00:00,553373,RU | Гидрактив
1,2022-07-28 08:00:00,613554,RU | Гидрактив
2,2022-07-26 21:00:00,536314,RU | Гидрактив
3,2022-07-19 19:00:00,230898,RU | Гидрактив
4,2022-07-27 03:00:00,215257,RU | Гидрактив
...,...,...,...
278,2022-07-21 18:00:00,353819,RU | Гидрактив
279,2022-07-26 13:00:00,742657,RU | Гидрактив
280,2022-07-19 09:00:00,374665,RU | Гидрактив
281,2022-07-22 06:00:00,226570,RU | Гидрактив


In [2]:
df.columns = ['datetime', 'shows', 'campaings']

test = df.sort_values('datetime')
test = test.drop(test.loc[test['shows'] == 0].index)\
            .drop(test.index.values[-5:])\
            .reset_index().drop('index', axis=1)
test['datetime'] = pd.to_datetime(test['datetime'])
test

,datetime,shows,campaings
0,2022-07-19 09:00:00,374665,RU | Гидрактив
1,2022-07-19 10:00:00,398972,RU | Гидрактив
2,2022-07-19 11:00:00,359010,RU | Гидрактив
3,2022-07-19 12:00:00,335606,RU | Гидрактив
4,2022-07-19 13:00:00,324781,RU | Гидрактив
...,...,...,...
232,2022-07-29 01:00:00,175877,RU | Гидрактив
233,2022-07-29 02:00:00,170458,RU | Гидрактив
234,2022-07-29 03:00:00,200582,RU | Гидрактив
235,2022-07-29 04:00:00,254294,RU | Гидрактив


In [5]:
from scipy.optimize import minimize
from cross_val import CVScore

x=[1,0,1]

timeseriesCV = CVScore(test['shows'])

opt = minimize(timeseriesCV.timeseriesCVscore, x0=x, method='TNC', 
                bounds=((0, 1), (0, 1), (0, 1)), options={'maxiter': 10000})

opt

     fun: 12637771310.657455
     jac: array([-8.75242038e+08,  5.43275102e+11, -0.00000000e+00])
 message: 'Local minimum reached (|pg| ~= 0)'
    nfev: 4
     nit: 1
  status: 0
 success: True
       x: array([1., 0., 1.])

In [14]:
df

,datetime,shows,campaings
0,2022-07-27 10:00:00,553373,RU | Гидрактив
1,2022-07-28 08:00:00,613554,RU | Гидрактив
2,2022-07-26 21:00:00,536314,RU | Гидрактив
3,2022-07-19 19:00:00,230898,RU | Гидрактив
4,2022-07-27 03:00:00,215257,RU | Гидрактив
...,...,...,...
278,2022-07-21 18:00:00,353819,RU | Гидрактив
279,2022-07-26 13:00:00,742657,RU | Гидрактив
280,2022-07-19 09:00:00,374665,RU | Гидрактив
281,2022-07-22 06:00:00,226570,RU | Гидрактив


# Build model & validation

In [6]:
hw = HoltWinters(test['shows'], slen=24, alpha=opt.x[0], 
                    beta=opt.x[1], gamma=opt.x[2], n_preds=0,
                    scaling_factor=2.56)
hw.triple_exponential_smoothing()

validation = pd.DataFrame(dict(shows=test['shows'],
                                datetime=test['datetime'],
                                predict=hw.result))
validation

validation['SSR'] = (validation['shows'] - validation['predict'])**2
validation['SST'] = (validation['shows'] - validation['shows'].mean())**2
check = validation['SSR'].sum()/validation['SST'].sum()
check

2.605776988434506e-06

# Forecast

In [7]:
hw_forecast = HoltWinters(test['shows'], slen=24, alpha=opt.x[0], 
                    beta=opt.x[1], gamma=opt.x[2], n_preds=10,
                    scaling_factor=2.56)
hw_forecast.triple_exponential_smoothing()

forecast_arr = []
for i in range(int(len(test)/3)):
    forecast_arr.append(test['datetime'].values[-1]+pd.Timedelta(hours=i))
forecast = pd.DataFrame(forecast_arr, columns=['datetime'])
forecast['forecast'] = hw_forecast.result[-len(forecast['datetime']):]

In [12]:
predict = pd.concat([test[['datetime', 'shows']], 
                        forecast[['datetime', 'forecast']]],
                    ignore_index=True)
predict.loc[predict['shows'].isna() == True]['forecast'].sum()

39171440.38888889

In [14]:
predict.to_csv('test.csv')

# Build plot

In [19]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=test['datetime'].values, 
                        y=test['shows'].values,
                        name=f'campaign {campaigns[0]}'))
fig.add_trace(go.Scatter(x=test['datetime'].values,
                            y=hw.result, name='predict'))
fig.add_trace(go.Scatter(x=forecast['datetime'].values, 
                            y=forecast['forecast'].values,
                            name=f'forecast for {50} values'))
fig.update_layout(
    font_family="Courier New",
    title_text=f'Forecast for campaign {campaigns[0]} with accurancy {round(100-check, 10)}%'
)
fig.show()

In [36]:
forecast

,datetime,forecast
0,2022-07-27 09:00:00,558199.499467
1,2022-07-27 10:00:00,568726.627182
2,2022-07-27 11:00:00,569447.553802
3,2022-07-27 12:00:00,570138.718981
4,2022-07-27 13:00:00,589215.546619
...,...,...
59,2022-07-29 20:00:00,512804.119229
60,2022-07-29 21:00:00,482743.116114
61,2022-07-29 22:00:00,400906.908393
62,2022-07-29 23:00:00,346289.967406


In [69]:
import plotly.graph_objects as go

forecastPeriod=10

forecast = exp.forecast(forecastPeriod)

fig = go.Figure()

fig.add_trace(go.Scatter(x=test['datetime'].values, 
                        y=test['shows'].values,
                        name=f'Campaigns {campaigns[0]}'))

fig.add_trace(go.Scatter(x=[test['datetime'].values[-1] + pd.Timedelta(hours=i) 
                                for i in range(forecastPeriod)],
                            y=forecast, 
                            name=f'Forecast for {forecastPeriod} hours'))

fig.show()